In [1]:
import pandas as pd

In [2]:
# specify directory that your data is in (this is relative)
path = 'INPUT/'

# file you want to read in
file = '2015 MCA-III Results Public Math.tsv'

# combine the two to create the path to the file
file_path = path + file

# read in file to append data to
data_df = pd.read_csv(file_path, sep='\t')

/Users/dreyco676/anaconda3/anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# look at top 5 rows
data_df.head()

,DataYear,DistrictCountyNumber,DistrictCountyName,DistrictNumber,DistrictType,DistrictName,SchoolNumber,SchoolName,ECSUNumber,EcoDevRgn,...,CountRefused,CountWrongGrade,GradeEnrollment,K12Enrollment,FilterThreshold,PublicSchool,SchoolCountyNumber,SchoolCountyName,SchoolClassification,SummaryLevel
0,14-15,1,Aitkin,1,1,AITKIN PUBLIC SCHOOL DISTRICT,0,All Schools,3,03,...,0,0,81,1184,9,Y,,NaN,0,district
1,14-15,1,Aitkin,1,1,AITKIN PUBLIC SCHOOL DISTRICT,0,All Schools,3,03,...,0,0,81,1184,9,Y,,NaN,0,district
2,14-15,1,Aitkin,1,1,AITKIN PUBLIC SCHOOL DISTRICT,0,All Schools,3,03,...,0,0,81,1184,9,Y,,NaN,0,district
3,14-15,1,Aitkin,1,1,AITKIN PUBLIC SCHOOL DISTRICT,0,All Schools,3,03,...,0,0,81,1184,9,Y,,NaN,0,district
4,14-15,1,Aitkin,1,1,AITKIN PUBLIC SCHOOL DISTRICT,0,All Schools,3,03,...,0,0,81,1184,9,Y,,NaN,0,district


In [26]:
# standardize capitalization
std_school_df = data_df['SchoolName'].str.lower()
# get list of unique names in the column
unique_names = pd.unique(std_school_df).tolist()

print(unique_names)

['all schools', 'aitkin secondary school', 'rippleside elementary', 'aitkin alternative learning program', 'armatage elementary', 'lake harriet lower elementary', 'bancroft elementary', 'barton open elementary', 'bethune elementary', 'burroughs elementary', 'field elementary', 'lake harriet upper school', 'hale elementary', 'howe elementary', 'lk nokomis comm-keewaydin campus', 'kenny elementary', 'kenwood elementary', 'loring elementary', 'lyndale elementary', 'jenny lind elementary', 'northrop elementary', 'pillsbury elementary', 'pratt elementary', 'seward elementary', 'sheridan elementary', 'waite park elementary', 'lk nokomis comm-wenonah campus', 'windom school', 'folwell arts magnet', 'jefferson elementary', 'dowling elementary', 'andersen community', 'sullivan elementary', 'pierre bottineau', 'anishinabe academy', 'marcy open elementary', 'bryn mawr elementary', 'green central park elementary', 'emerson elementary', 'lucy laney @ cleveland park elem.', 'hall international', 'ne

In [4]:
import googlemaps
# You'll need your own API key
# https://developers.google.com/maps/signup?hl=en

# Read in my API key
with open("api_key.txt", "r") as file:
    api_key = file.read()
    
# start gmaps api
gmaps = googlemaps.Client(key=api_key)

In [28]:
# Good to keep API Rate Limits in mind.
# https://developers.google.com/places/web-service/usage
# 10 per second, 1000 a day
# 150,000 if you verify with CC that you are not a bot
print(len(unique_names))

1892


In [5]:
# let's look at just doing one at a time before we run a whole list
# use google maps places API to get geolocation information 
loc = 'AITKIN SECONDARY SCHOOL'
gmaps_json = gmaps.places(loc, location='Minnesota', radius=1000, types='school')

# use pretty print library to have nicer looking output
import pprint
pprint.pprint(gmaps_json)

Timeout: 

In [17]:
# let's grab the fields we care about
results = gmaps_json["results"][0]

std_name = results['name']
lat = results['geometry']['location']['lat']
lng = results['geometry']['location']['lng']
std_address = results['formatted_address']

print(std_name, lat, lng, std_address)

Aitkin Secondary School 46.5336663 -93.71121749999999 306 2nd St NW, Aitkin, MN 56431, United States


In [18]:
# tag the address to get nice filtering criteria like zip and city
import usaddress
parsed_address = usaddress.tag(std_address)

pprint.pprint(parsed_address)

({'AddressNumber': '306',
  'StreetName': '2nd',
  'StreetNamePostType': 'St',
  'StreetNamePostDirectional': 'NW',
  'PlaceName': 'Aitkin',
  'StateName': 'MN',
  'ZipCode': '56431',
  'CountryName': 'United States'},
 'Street Address')


In [19]:
# select the pieces we care about
city = parsed_address[0]['PlaceName']
state = parsed_address[0]['StateName']

print(city, state)

Aitkin MN 56431


In [21]:
# let's put this all together to make a single record in a table
# important to keep the original name searched, use this to join back
df = pd.DataFrame([[loc, std_name, lat, lng, city, state]], 
                  columns=['Search_Term', 'Name', 'Latitude', 'Longitude', 'City', 'State'])

print(df)

               Search_Term                     Name   Latitude  Longitude  \
0  AITKIN SECONDARY SCHOOL  Aitkin Secondary School  46.533666 -93.711217   

     City State ZipCode  
0  Aitkin    MN   56431  


In [ ]:
import googlemaps
import usaddress

# ok let's try this across the whole list
geoloc_df = pd.DataFrame()
for loc in unique_names:
    try:
        gmaps_json = gmaps.places(loc, location='Minnesota', radius=1000, types='school')
        print(gmaps_json)
        # parse json
        results = gmaps_json["results"][0]
        std_name = results['name']
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
        std_address = results['formatted_address']
        # parse the address
        parsed_address = usaddress.tag(std_address)
        city = parsed_address[0]['PlaceName']
        state = parsed_address[0]['StateName']
        df = pd.DataFrame([[loc, std_name, lat, lng, city, state]], 
                  columns=['Raw_Name', 'Name', 'Latitude', 'Longitude', 'City', 'State'])
        geoloc_df = geoloc_df.append(df)
    # if something fails, just skip. bad coding
    except:
        print(loc, 'Failed.')


all schools Failed.
aitkin secondary school Failed.
rippleside elementary Failed.
aitkin alternative learning program Failed.
armatage elementary Failed.
lake harriet lower elementary

In [ ]:
# save data so I don't bore you in the talk
geoloc_df.to_csv('tutorial_cached.txt', sep='\t', index=False, mode='w')

In [7]:
# load data so I don't bore you in the talk
geoloc_df = pd.read_csv('tutorial_cached.txt', sep='\t')

print(geoloc_df.head())

                           Raw_Name                                Name  \
0                              test                                test   
1          little canada elementary     Little Canada Elementary School   
2                            attain                              Attain   
3  west st. paul-mendota hts.-eagan  Independent School District No 197   
4    lakeville area learning center      Lakeville Area Learning Center   

    Latitude   Longitude             City State  
0  23.000000  343.000000       Eau Claire    WI  
1  45.024745  -93.083058    Little Canada    MN  
2  38.926405  -77.225298           McLean    VA  
3  44.887310  -93.108026  Mendota Heights    MN  
4  44.646357  -93.243841        Lakeville    MN  


In [14]:
# now let's append the geolocation data to the original data
# data might have wholes in it so we'll do a left join (keep everything in original + matches in geolocation)

result_df = pd.merge(data_df, geoloc_df, how='left', left_on='SchoolName', right_on='Raw_Name')

# ideally here you would look to see how many don't have matches and may try to fix those by hand
matched_df = result_df.dropna(subset=['Name'], how='all')


Empty DataFrame
Columns: [DataYear, DistrictCountyNumber, DistrictCountyName, DistrictNumber, DistrictType, DistrictName, SchoolNumber, SchoolName, ECSUNumber, EcoDevRgn, Testname, TestDate, Grade, Subject, ReportOrder, ReportCategory, ReportDescription, Filtered, CountTested, CountLevel1, CountLevel2, CountLevel3, CountLevel4, PercentLevel1, PercentLevel2, PercentLevel3, PercentLevel4, AverageScore, NOPSAverageScore, ALGSAverageScore, GMSAverageScore, DAPSAverageScore, VSSAverageScore, Stddev, StdErrorOfMean, ConfidenceInterval, CountAbsent, CountInvalid, CountMedExempt, CountNotComplete, CountPSO, CountRefused, CountWrongGrade, GradeEnrollment, K12Enrollment, FilterThreshold, PublicSchool, SchoolCountyNumber, SchoolCountyName, SchoolClassification, SummaryLevel, Raw_Name, Name, Latitude, Longitude, City, State]
Index: []

[0 rows x 57 columns]
